In [1]:
import cv2
import numpy as np
from numpy.core.numeric import True_
from numpy.lib.arraysetops import unique
import pandas as pd
# import glob
# import os
from pandas import json_normalize
from os import getcwd, path
from yaml import SafeLoader, load
import datetime
import matplotlib.pyplot as plt
import yaml
import shutil
import json

from video_utils_virat import VideoUtils

%matplotlib

Using matplotlib backend: TkAgg


## Dataset Selection

In [2]:

# setup
dataset_dir_path = './datasets/VIRAT/'
image_ext = '.jpg'
video_max_frames = 2000

#video
video_src_path = dataset_dir_path + 'Videos/Ground/'
video_ext = '.mp4'
video_name = 'VIRAT_S_000203_06_001206_001266'
video_name_orig = video_name + video_ext
video_dest_path = './' + video_name + '/'
video_name_new = 'annotated_' + video_name + '.avi'#video_ext

# annotations
annotations_path = dataset_dir_path + 'viratannotations-master/train/'
ann_activities_file = annotations_path + video_name + '.activities.yml'
ann_geom_file = annotations_path + video_name + '.geom.yml'
ann_regions_file = annotations_path + video_name + '.regions.yml'
ann_types_file = annotations_path + video_name + '.types.yml'

## Creating annotations dataframe

In [3]:
# using annotations:
print("Loading annotations...")
with open(ann_geom_file) as yaml_file:
    yaml_contents = load(yaml_file, Loader=SafeLoader)
yaml_df = json_normalize(yaml_contents)

df_bbox = yaml_df[['geom.id1','geom.ts0','geom.ts1','geom.g0']].dropna().reset_index()
df_bbox.rename(columns={'geom.id1': 'object id', 'geom.ts0': 'frame_id','geom.ts1': 'time_sec', 'geom.g0': 'bbox'}, inplace=True)
df_bbox['bbox'] = df_bbox['bbox'].str.split()
df_tmp = pd.DataFrame(df_bbox['bbox'].to_list(), columns = ['bb_Left', 'bb_Top', 'bb_Right', 'bb_Bottom'])
df_bbox = pd.concat([df_bbox, df_tmp], axis=1).drop(columns=['bbox'])
df_bbox.to_csv('df_bbox.csv')

Loading annotations...


In [4]:
df_bbox = pd.read_csv('df_bbox.csv')
df_bbox.drop(['Unnamed: 0'], axis=1)

,index,object id,frame_id,time_sec,bb_Left,bb_Top,bb_Right,bb_Bottom
0,4,0.0,0.0,0.000000,0,328,24,361
1,5,0.0,1.0,0.033333,0,328,24,361
2,6,0.0,2.0,0.066667,0,328,24,361
3,7,0.0,3.0,0.100000,0,328,24,361
4,8,0.0,4.0,0.133333,0,328,24,361
...,...,...,...,...,...,...,...,...
112255,112259,5030.0,1776.0,59.200000,72,423,95,487
112256,112260,5030.0,1777.0,59.233333,72,423,95,487
112257,112261,5030.0,1778.0,59.266667,72,423,95,487
112258,112262,5030.0,1779.0,59.300000,72,423,95,487


## Annotate Video

In [5]:
from video_utils_virat import VideoUtils
from video_utils_virat import DirectoryUtils

# vUtils = VideoUtils(categoriesDict) 
vUtils = VideoUtils() 
drUtils = DirectoryUtils()


In [6]:
# vUtils.AnnotateVideo(video_dest_path, video_src_path + video_name_orig, video_name_new, df_bbox['bbox'].to_numpy(), fps, 2000)

# drUtils.ClearFileType(video_dest_path,".jpg")

In [7]:
vUtils = VideoUtils() 

def AddAllFrameAnnotations(img, df_ann, box_thickness):
    for j in range(len(df_ann)):
        img = vUtils.AddSingleAnnotation(img, df_ann.iloc[j].to_dict(), box_thickness)
    return img


def AnnotateVideo(output_dir, orig_v_full_path, new_v_filename, df_ann, start_time = 0, duration = []):
    # Open original video
    vidcap = cv2.VideoCapture(orig_v_full_path)
    fps = vidcap.get(5) #CV_CAP_PROP_FPS = Frame rate.
    total_frames = vidcap.get(7) # CV_CAP_PROP_FRAME_COUNT = Number of frames in the video file.
    print(f'Total frames in video: {total_frames} @ {fps} frames/sec')

    count = int(start_time * fps)
    if(bool(duration)):
        end_count = int((duration + start_time) * fps)
    else:
        end_count = total_frames - count
        print('running full video')


    vidcap.set(1, count) #setting CV_CAP_PROP_POS_FRAMES at count 

    images = []
    img_array = np.array(images)



    # Create directory to store new video
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    os.makedirs(output_dir)

    while (True):
        success, img = vidcap.read()
        if(success):

            # Creating a new image name
            tmp_img_name = output_dir + "frame%d.jpg" % count
            # Added new image to array of images that will construct the final video
            img_array = np.append(img_array, tmp_img_name)

            if((count % 25) == 0):
                # print(f'Created frame id {count}, {count/fps} sec in video')
                print("Created frame id %2d, %5.2f sec in video" % (count, count/fps))


            # All bounding box info for this frame
            # Add all annotations to the frame
            df_img = df_bbox[df_ann['frame_id']==count]
            img = AddAllFrameAnnotations(img, df_img, 2)

            # save the image with all its annotations
            cv2.imwrite(tmp_img_name, img)
        
            count += 1
            
            if(count > end_count):
                break

        else:
            break

    vidcap.release()  # done with original video

    full_filename = output_dir + new_v_filename
    if os.path.exists(full_filename):
        os.remove(full_filename)

    vUtils.CreateVideo(img_array, fps, full_filename)
    cv2.destroyAllWindows()

In [8]:
start_time = 0
AnnotateVideo(video_dest_path, video_src_path + video_name_orig, video_name_new, df_bbox, start_time)

drUtils.ClearFileType(video_dest_path,".jpg")

Total frames in video: 1781.0 @ 30.0 frames/sec
running full video
Created frame id  0,  0.00 sec in video
Created frame id 25,  0.83 sec in video
Created frame id 50,  1.67 sec in video
Created frame id 75,  2.50 sec in video
Created frame id 100,  3.33 sec in video
Created frame id 125,  4.17 sec in video
Created frame id 150,  5.00 sec in video
Created frame id 175,  5.83 sec in video
Created frame id 200,  6.67 sec in video
Created frame id 225,  7.50 sec in video
Created frame id 250,  8.33 sec in video
Created frame id 275,  9.17 sec in video
Created frame id 300, 10.00 sec in video
Created frame id 325, 10.83 sec in video
Created frame id 350, 11.67 sec in video
Created frame id 375, 12.50 sec in video
Created frame id 400, 13.33 sec in video
Created frame id 425, 14.17 sec in video
Created frame id 450, 15.00 sec in video
Created frame id 475, 15.83 sec in video
Created frame id 500, 16.67 sec in video
Created frame id 525, 17.50 sec in video
Created frame id 550, 18.33 sec in 

In [9]:
# bb_data = df_bbox['bbox'].to_numpy()
# count = 0
# max_ann = 0
# ann_cnt = 0
# df_center = pd.DataFrame([])
# df2  = pd.DataFrame([])
# frame_cnt = len(bb_data)
# for i in range(frame_cnt):
#   ann_cnt = len(bb_data[i])
#   lst_x = np.array([])
#   lst_y = np.array([])

#   for j in range(ann_cnt):
#     bbox = bb_data[i][j]
#     top = bbox['top']
#     left = bbox['left']
#     height = bbox['height']
#     width = bbox['width']
#     categories = bbox['class']
#     bottom = top + height
#     right = left + width
#     center_x = int(left + width/2)
#     center_y = int(top + height/2)
#     center = (center_x, center_y)

#     lst_x = np.append(lst_x, center_x)
#     lst_y = np.append(lst_y, center_y)
#     count += 1
#     max_ann = max(max_ann, ann_cnt)

#   lst_tuple = list(zip(lst_x,lst_y))
#   print(count, ann_cnt, max_ann, lst_tuple)
#   #   df2[i] = bbox
#   # df_center = df_center.append(df2, ignore_index = True)
# # print(i, df_center)

# # velociy_pix [pixel/frame]
# # frameRate   [frame/sec]
# # scale       [m/pixel]
# # velocity = velociy_pix * frameRate * scale; #   [m/sec] = [pixel/frame] * [frame/sec] * [m/pixel]


In [10]:
# import datashader as ds
# import pandas as pd
# import colorcet
# # df  = pd.read_csv('census.csv')
# cvs = ds.Canvas(plot_width=850, plot_height=500)
# agg = cvs.points(df_json, 'longitude', 'latitude')
# img = ds.tf.shade(agg, cmap=colorcet.fire, how='log')